In [4]:
# data path
datadownloadpath_Train = './datadownload/MnMCardiac/Training/'
datadownloadpath_Val = './datadownload/MnMCardiac/Validation/'
datadownloadpath_Test = './datadownload/MnMCardiac/Testing/'
datasavepath = './datapreprocessed/MnMCardiac/'
metadata = './datadownload/MnMCardiac/211230_M&Ms_Dataset_information_diagnosis_opendataset.csv'

In [2]:
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd
import nibabel as nib
from scipy.ndimage import label, binary_fill_holes
import os

In [3]:
if os.path.exists(datasavepath + 'Image') == False:
    os.mkdir(datasavepath)
    os.mkdir(datasavepath + 'alldata')
    os.mkdir(datasavepath + 'alllabels')
    os.mkdir(datasavepath + 'Image')
    os.mkdir(datasavepath + 'Label')
    os.mkdir(datasavepath + 'Image_wohdr')
    os.mkdir(datasavepath + 'Label_wohdr')
    os.mkdir(datasavepath + 'processed')
    os.mkdir(datasavepath + 'mask')

In [4]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    resample.SetUseNearestNeighborExtrapolator(True)
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj

In [41]:
# just use this split.
imgs_train = glob.glob(datadownloadpath_Train + "*/*/*_sa.nii.gz")
imgs_train = [ fid for fid in sorted(imgs_train) ]
imgs_val = glob.glob(datadownloadpath_Val + "*/*_sa.nii.gz")
imgs_val = [ fid for fid in sorted(imgs_val) ]
imgs_test = glob.glob(datadownloadpath_Test + "*/*_sa.nii.gz")
imgs_test = [ fid for fid in sorted(imgs_test) ]
imgs = imgs_train + imgs_val + imgs_test

segs_train = glob.glob(datadownloadpath_Train + "*/*/*_sa_gt.nii.gz")
segs_train = [ fid for fid in sorted(segs_train) ]
segs_val = glob.glob(datadownloadpath_Val + "*/*_sa_gt.nii.gz")
segs_val = [ fid for fid in sorted(segs_val) ]
segs_test = glob.glob(datadownloadpath_Test + "*/*_sa_gt.nii.gz")
segs_test = [ fid for fid in sorted(segs_test) ]
segs = segs_train + segs_val + segs_test


pids = [pid.split("/")[-1].split("_")[0] for pid in imgs]

In [42]:
imgs

['./datadownload/MnMCardiac/Training/Labeled/A0S9V9/A0S9V9_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1D0Q7/A1D0Q7_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1D9Z7/A1D9Z7_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1E9Q1/A1E9Q1_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1O8Z3/A1O8Z3_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A2C0I1/A2C0I1_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A2N8V0/A2N8V0_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A3B7E5/A3B7E5_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A3H1O5/A3H1O5_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4B5U4/A4B5U4_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4J4S4/A4J4S4_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4U9V5/A4U9V5_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A5E0T8/A5E0T8_sa.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A6B5G9/A6B5G9_sa.nii.gz',
 './da

In [43]:
segs

['./datadownload/MnMCardiac/Training/Labeled/A0S9V9/A0S9V9_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1D0Q7/A1D0Q7_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1D9Z7/A1D9Z7_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1E9Q1/A1E9Q1_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A1O8Z3/A1O8Z3_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A2C0I1/A2C0I1_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A2N8V0/A2N8V0_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A3B7E5/A3B7E5_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A3H1O5/A3H1O5_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4B5U4/A4B5U4_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4J4S4/A4J4S4_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A4U9V5/A4U9V5_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/Labeled/A5E0T8/A5E0T8_sa_gt.nii.gz',
 './datadownload/MnMCardiac/Training/L

In [44]:
pids

['A0S9V9',
 'A1D0Q7',
 'A1D9Z7',
 'A1E9Q1',
 'A1O8Z3',
 'A2C0I1',
 'A2N8V0',
 'A3B7E5',
 'A3H1O5',
 'A4B5U4',
 'A4J4S4',
 'A4U9V5',
 'A5E0T8',
 'A6B5G9',
 'A6D5F9',
 'A6M1Q7',
 'A7D9L8',
 'A7G0P5',
 'A7M7P8',
 'A7O4T6',
 'A8C9U8',
 'A8E1F4',
 'A9C5P4',
 'A9E3G9',
 'A9J5Q7',
 'A9J8W7',
 'B0I2Z0',
 'B0N3W8',
 'B2C2Z7',
 'B2D9M2',
 'B2D9O2',
 'B2F4K5',
 'B2G5R2',
 'B3D0N1',
 'B3O1S0',
 'B3P3R1',
 'B4O3V3',
 'B6D0U7',
 'B8H5H6',
 'B8J7R4',
 'B9E0Q1',
 'B9O1Q0',
 'C0K1P0',
 'C0S7W0',
 'C1G5Q0',
 'C1K8P5',
 'C2J0K3',
 'C2L5P7',
 'C2M6P8',
 'C3I2K3',
 'C4R8T7',
 'C4S8W9',
 'C5M4S2',
 'C6J5P1',
 'C8P3S7',
 'D0H9I4',
 'D0R0R9',
 'D1J5P6',
 'D1L4Q9',
 'D1M1S6',
 'D3D4Y5',
 'D3F3O5',
 'D3F9H9',
 'D3O9U9',
 'D4M3Q2',
 'D4N6W6',
 'D6H6O2',
 'D8E4F4',
 'D9L1Z3',
 'E0M3U7',
 'E0O0S0',
 'E3T0Z2',
 'E4M2Q7',
 'E4W8Z7',
 'E5E6O8',
 'E5F5V7',
 'E9H1U4',
 'E9H2K7',
 'E9L1W5',
 'E9V4Z8',
 'F0J2R8',
 'F1F3I6',
 'F2H5S1',
 'F3G5K5',
 'F4K3S1',
 'F5I9Q2',
 'F8N2S1',
 'G0H4J3',
 'G0I6P3',
 'G1N6S7',
 'G2J1M5',

## Step 0, Generate slices

In [10]:
for (imgfile, lblfile, pid) in zip(imgs, segs, pids):
    print('Processing Case ' + pid)
    lblload = nib.load(lblfile)
    lbl = lblload.get_fdata()
    
    for n_frame in range(lbl.shape[-1]):
        
        if np.sum(lbl[:, :, :, n_frame]) > 0: # this slice is labeled,
            
        
            Imgload = nib.load(imgfile)
            Imgc1 = Imgload.get_fdata() 

            affine_origin = Imgload.affine
#             affine_origin = np.eye(4)
            newLabelImg = nib.Nifti1Image(Imgc1[:, :, :, n_frame], affine_origin)
            newLabelImg.set_qform(newLabelImg.get_qform())
            newLabelImg.set_sform(newLabelImg.get_sform())
            imgsavepath = datasavepath + 'alldata/' + pid + '_' + str(n_frame) + '.nii.gz'
            nib.save(newLabelImg, imgsavepath)

            newLabelLbl = nib.Nifti1Image(lbl[:, :, :, n_frame], affine_origin)
            newLabelLbl.set_qform(newLabelLbl.get_qform())
            newLabelLbl.set_sform(newLabelLbl.get_sform())
            lblsavepath = datasavepath + 'alllabels/' + pid + '_' + str(n_frame) + '.nii.gz'
            nib.save(newLabelLbl, lblsavepath)
    
    print('Data shape ' + str(np.array(lbl[:, :, :, n_frame].shape)))
    print('Case ' + pid + ' saved')

Processing Case A0S9V9
Data shape [216 256  13]
Case A0S9V9 saved
Processing Case A1D0Q7
Data shape [320 320  10]
Case A1D0Q7 saved
Processing Case A1D9Z7
Data shape [208 256  13]
Case A1D9Z7 saved
Processing Case A1E9Q1
Data shape [208 256  13]
Case A1E9Q1 saved
Processing Case A1O8Z3
Data shape [256 256  11]
Case A1O8Z3 saved
Processing Case A2C0I1
Data shape [208 256  12]
Case A2C0I1 saved
Processing Case A2N8V0
Data shape [196 240  11]
Case A2N8V0 saved
Processing Case A3B7E5
Data shape [320 320  10]
Case A3B7E5 saved
Processing Case A3H1O5
Data shape [196 240  11]
Case A3H1O5 saved
Processing Case A4B5U4
Data shape [196 240  14]
Case A4B5U4 saved
Processing Case A4J4S4
Data shape [208 256  11]
Case A4J4S4 saved
Processing Case A4U9V5
Data shape [196 240  12]
Case A4U9V5 saved
Processing Case A5E0T8
Data shape [256 256  11]
Case A5E0T8 saved
Processing Case A6B5G9
Data shape [196 240  12]
Case A6B5G9 saved
Processing Case A6D5F9
Data shape [216 256  13]
Case A6D5F9 saved
Processing

Data shape [320 320   9]
Case L1Q1Z5 saved
Processing Case L1Q9V8
Data shape [240 196  12]
Case L1Q9V8 saved
Processing Case L4Q2U3
Data shape [240 196  12]
Case L4Q2U3 saved
Processing Case L5Q6T7
Data shape [320 320  10]
Case L5Q6T7 saved
Processing Case M0P8U8
Data shape [288 288  12]
Case M0P8U8 saved
Processing Case M1R4S1
Data shape [208 256  10]
Case M1R4S1 saved
Processing Case M2P1R1
Data shape [196 240  10]
Case M2P1R1 saved
Processing Case M4P7Q6
Data shape [384 384   8]
Case M4P7Q6 saved
Processing Case N1P8Q9
Data shape [288 288   8]
Case N1P8Q9 saved
Processing Case N5S7Y1
Data shape [196 240  12]
Case N5S7Y1 saved
Processing Case N7V9W9
Data shape [256 256  12]
Case N7V9W9 saved
Processing Case N8N9U0
Data shape [196 240  14]
Case N8N9U0 saved
Processing Case O0S9V7
Data shape [196 240  12]
Case O0S9V7 saved
Processing Case O3R8Y5
Data shape [320 320  10]
Case O3R8Y5 saved
Processing Case P0S5Y0
Data shape [208 256  11]
Case P0S5Y0 saved
Processing Case P5R1Y4
Data shape

Data shape [256 256  12]
Case C7M6W0 saved
Processing Case C8O0P2
Data shape [256 256  10]
Case C8O0P2 saved
Processing Case D1H2O9
Data shape [256 256  13]
Case D1H2O9 saved
Processing Case D1L6T4
Data shape [512 440  12]
Case D1L6T4 saved
Processing Case D3K5Q2
Data shape [256 256   8]
Case D3K5Q2 saved
Processing Case D3Q0W9
Data shape [288 288  10]
Case D3Q0W9 saved
Processing Case D6E9U8
Data shape [320 320  10]
Case D6E9U8 saved
Processing Case D9F5P1
Data shape [208 256   9]
Case D9F5P1 saved
Processing Case D9I8O7
Data shape [288 288  12]
Case D9I8O7 saved
Processing Case E0J2Z9
Data shape [512 440  14]
Case E0J2Z9 saved
Processing Case E0J7L9
Data shape [208 256   9]
Case E0J7L9 saved
Processing Case E1L7M3
Data shape [480 436  13]
Case E1L7M3 saved
Processing Case E3F5U2
Data shape [256 256  11]
Case E3F5U2 saved
Processing Case E3L8U8
Data shape [288 288  12]
Case E3L8U8 saved
Processing Case E4H7L4
Data shape [512 440  15]
Case E4H7L4 saved
Processing Case E4I9O7
Data shape

## Step 1, Data resample

In [11]:
# get the volume list
imgs_volume = glob.glob(datasavepath + 'alldata/' + "*.nii.gz")
imgs_volume = [ fid for fid in sorted(imgs_volume) ]

segs_volume = glob.glob(datasavepath + 'alllabels/' + "*.nii.gz")
segs_volume = [ fid for fid in sorted(segs_volume) ]

pids_volume = [pids_volume.split("/")[-1].split(".")[0] for pids_volume in imgs_volume]

In [12]:
imgs_volume

['./datapreprocessed/MnMCardiac/alldata/A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A2C0I1_7.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A2E3W4_23.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A2E3W4_8.nii.gz',
 './datapreprocessed/MnMCardiac/alldata/A2H5K9_29.nii.gz',
 './da

In [13]:
segs_volume

['./datapreprocessed/MnMCardiac/alllabels/A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A2C0I1_7.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A2E3W4_23.nii.gz',
 './datapreprocessed/MnMCardiac/alllabels/A2E3W4_8.nii.gz',
 './datapreprocessed/MnMCardiac/a

In [14]:
pids_volume

['A0S9V9_0',
 'A0S9V9_9',
 'A1D0Q7_0',
 'A1D0Q7_9',
 'A1D9Z7_11',
 'A1D9Z7_22',
 'A1E9Q1_0',
 'A1E9Q1_9',
 'A1K2P5_11',
 'A1K2P5_33',
 'A1O8Z3_10',
 'A1O8Z3_23',
 'A2C0I1_0',
 'A2C0I1_7',
 'A2E3W4_23',
 'A2E3W4_8',
 'A2H5K9_29',
 'A2H5K9_8',
 'A2L1N6_0',
 'A2L1N6_12',
 'A2N8V0_0',
 'A2N8V0_9',
 'A3B7E5_12',
 'A3B7E5_29',
 'A3H1O5_0',
 'A3H1O5_12',
 'A3H5R1_24',
 'A3H5R1_6',
 'A3P9V7_13',
 'A3P9V7_27',
 'A4A8V9_0',
 'A4A8V9_10',
 'A4B5U4_0',
 'A4B5U4_10',
 'A4B9O6_0',
 'A4B9O6_11',
 'A4J4S4_0',
 'A4J4S4_7',
 'A4K8R4_11',
 'A4K8R4_24',
 'A4R4T0_21',
 'A4R4T0_8',
 'A4U9V5_0',
 'A4U9V5_8',
 'A5C2D2_24',
 'A5C2D2_9',
 'A5D0G0_0',
 'A5D0G0_10',
 'A5E0T8_24',
 'A5E0T8_8',
 'A5H1Q2_0',
 'A5H1Q2_9',
 'A5P5W0_10',
 'A5P5W0_33',
 'A5Q1W8_10',
 'A5Q1W8_27',
 'A6A8H0_27',
 'A6A8H0_8',
 'A6B5G9_0',
 'A6B5G9_11',
 'A6B7Y4_1',
 'A6B7Y4_10',
 'A6D5F9_0',
 'A6D5F9_11',
 'A6J0Y2_13',
 'A6J0Y2_27',
 'A6M1Q7_11',
 'A6M1Q7_29',
 'A7D9L8_0',
 'A7D9L8_11',
 'A7E4J0_1',
 'A7E4J0_12',
 'A7F4G2_10',
 'A7F4G2_25'

In [15]:
targetspacing = [1.25, 1.25, 10.]

In [16]:
for (imgfile, lblfile, pid) in zip(imgs_volume, segs_volume, pids_volume):
    savename = 'Cardiac_' + pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    seg_obj = sitk.ReadImage( lblfile )
    img_spa_ori = img_obj.GetSpacing()
    seg_obj.SetSpacing(img_spa_ori)
    res_img_o = resample_by_res(img_obj, targetspacing, interpolator = sitk.sitkLinear,
                                    logging = True)
    res_lb_o = resample_lb_by_res(seg_obj, targetspacing, interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    sitk.WriteImage(res_img_o, datasavepath + 'Image/' + savename, True) 
    sitk.WriteImage(res_lb_o, datasavepath + 'Label/' + savename, True) 

Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Label values: [0. 1. 2. 3.]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]
Label values: [0. 1. 2. 3.]
Spacing: (1.328125, 1.328125, 9.519999504089355) -> [1.25, 1.25, 10.0]
Size (216, 256, 13) -> [229.5        272.          12.37599936]

Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 12) -> [208.  256.    9.6]
Spacing: (1.484400391578

Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Label values: [0. 1. 2. 3.]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Spacing: (1.4166666269302368, 1.4166666269302368, 9.600000381469727) -> [1.25, 1.25, 10.0]
Size (196, 240, 11) -> [222.1333271  271.99999237  10.56000042]
Label values: [0. 1. 2. 3.]
Spacing: (1.41

Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.01599312  11.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.01599312  11.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.01599312  11.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.01599312  11.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 11) -> [325.83679199 280.01599312  11.        ]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 11) -> [208.  256.    8.8]
Label values: [0. 1. 2. 3.]
Spacing: (1.25, 1.25, 8.0) -> [1.25, 1.25, 10.0]
Size (208, 256, 11) -> [208.  256.    8.8]
Spacing: (1.25, 1.

Spacing: (1.1986300945281982, 1.1986300945281982, 10.0) -> [1.25, 1.25, 10.0]
Size (352, 352, 10) -> [337.53423462 337.53423462  10.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (1.1986300945281982, 1.1986300945281982, 10.0) -> [1.25, 1.25, 10.0]
Size (352, 352, 10) -> [337.53423462 337.53423462  10.        ]
Spacing: (1.1986300945281982, 1.1986300945281982, 10.0) -> [1.25, 1.25, 10.0]
Size (352, 352, 10) -> [337.53423462 337.53423462  10.        ]
Spacing: (1.1986300945281982, 1.1986300945281982, 10.0) -> [1.25, 1.25, 10.0]
Size (352, 352, 10) -> [337.53423462 337.53423462  10.        ]
Spacing: (1.1986300945281982, 1.1986300945281982, 10.0) -> [1.25, 1.25, 10.0]
Size (352, 352, 10) -> [337.53423462 337.53423462  10.        ]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 11) -> [304. 304.  11.]
Label values: [0. 1. 2. 3.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 11) -> [304. 304.  11.]
Spacin

...

Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Label values: [0. 1. 2. 3.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Label values: [0. 1. 2. 3.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304. 304.  10.]
Spacing: (1.484375, 1.484375, 10.0) -> [1.25, 1.25, 10.0]
Size

Spacing: (1.4615384340286255, 1.4615384340286255, 10.0) -> [1.25, 1.25, 10.0]
Size (288, 288, 11) -> [336.7384552 336.7384552  11.       ]
Label values: [0. 1. 2. 3.]
Spacing: (1.4615384340286255, 1.4615384340286255, 10.0) -> [1.25, 1.25, 10.0]
Size (288, 288, 11) -> [336.7384552 336.7384552  11.       ]
Spacing: (1.4615384340286255, 1.4615384340286255, 10.0) -> [1.25, 1.25, 10.0]
Size (288, 288, 11) -> [336.7384552 336.7384552  11.       ]
Spacing: (1.4615384340286255, 1.4615384340286255, 10.0) -> [1.25, 1.25, 10.0]
Size (288, 288, 11) -> [336.7384552 336.7384552  11.       ]
Spacing: (1.4615384340286255, 1.4615384340286255, 10.0) -> [1.25, 1.25, 10.0]
Size (288, 288, 11) -> [336.7384552 336.7384552  11.       ]
Spacing: (1.484399437904358, 1.4844000339508057, 10.0) -> [1.25, 1.25, 10.0]
Size (256, 256, 10) -> [304.00500488 304.00512695  10.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (1.484399437904358, 1.4844000339508057, 10.0) -> [1.25, 1.25, 10.0]

Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 14) -> [325.83679199 280.01599312  14.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765

Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (1.1666666269302368, 1.1666666269302368, 9.59999942779541) -> [1.25, 1.25, 10.0]
Size (240, 196, 14) -> [223.99999237 182.9333271   13.4399992 ]
Label values: [0. 1. 2. 3.]
Spacing: (1.1666666269302368, 1.1666666269302368

Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Spacing: (0.7954999804496765, 0.7954999804496765, 10.0) -> [1.25, 1.25, 10.0]
Size (512, 440, 12) -> [325.83679199 280.01599312  12.        ]
Label values: [0.         1.00000002 2.00000004 3.00000006]
Spacing: (0.7954999804496765

### Cleaning the orientation. (for visualization with itk-snap)

In [5]:
# get the volume list
imgs_volume_resampled = glob.glob(datasavepath + 'Image/' + "*.nii.gz")
imgs_volume_resampled = [ fid for fid in sorted(imgs_volume_resampled) ]

segs_volume_resampled = glob.glob(datasavepath + 'Label/' + "*.nii.gz")
segs_volume_resampled = [ fid for fid in sorted(segs_volume_resampled) ]

pids_volume_resampled = [pids_volume_resampled.split("/")[-1].split(".")[0] for pids_volume_resampled in imgs_volume_resampled]

In [6]:
imgs_volume_resampled

['./datapreprocessed/MnMCardiac/Image/Cardiac_A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A2C0I1_7.nii.gz',
 './datapreprocessed/MnMCardiac/Image/Cardiac_A2E3W4_23.nii.gz',
 './datapreprocessed/MnMCardiac/I

In [8]:
segs_volume_resampled

['./datapreprocessed/MnMCardiac/Label/Cardiac_A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A2C0I1_7.nii.gz',
 './datapreprocessed/MnMCardiac/Label/Cardiac_A2E3W4_23.nii.gz',
 './datapreprocessed/MnMCardiac/L

In [9]:
pids_volume_resampled

['Cardiac_A0S9V9_0',
 'Cardiac_A0S9V9_9',
 'Cardiac_A1D0Q7_0',
 'Cardiac_A1D0Q7_9',
 'Cardiac_A1D9Z7_11',
 'Cardiac_A1D9Z7_22',
 'Cardiac_A1E9Q1_0',
 'Cardiac_A1E9Q1_9',
 'Cardiac_A1K2P5_11',
 'Cardiac_A1K2P5_33',
 'Cardiac_A1O8Z3_10',
 'Cardiac_A1O8Z3_23',
 'Cardiac_A2C0I1_0',
 'Cardiac_A2C0I1_7',
 'Cardiac_A2E3W4_23',
 'Cardiac_A2E3W4_8',
 'Cardiac_A2H5K9_29',
 'Cardiac_A2H5K9_8',
 'Cardiac_A2L1N6_0',
 'Cardiac_A2L1N6_12',
 'Cardiac_A2N8V0_0',
 'Cardiac_A2N8V0_9',
 'Cardiac_A3B7E5_12',
 'Cardiac_A3B7E5_29',
 'Cardiac_A3H1O5_0',
 'Cardiac_A3H1O5_12',
 'Cardiac_A3H5R1_24',
 'Cardiac_A3H5R1_6',
 'Cardiac_A3P9V7_13',
 'Cardiac_A3P9V7_27',
 'Cardiac_A4A8V9_0',
 'Cardiac_A4A8V9_10',
 'Cardiac_A4B5U4_0',
 'Cardiac_A4B5U4_10',
 'Cardiac_A4B9O6_0',
 'Cardiac_A4B9O6_11',
 'Cardiac_A4J4S4_0',
 'Cardiac_A4J4S4_7',
 'Cardiac_A4K8R4_11',
 'Cardiac_A4K8R4_24',
 'Cardiac_A4R4T0_21',
 'Cardiac_A4R4T0_8',
 'Cardiac_A4U9V5_0',
 'Cardiac_A4U9V5_8',
 'Cardiac_A5C2D2_24',
 'Cardiac_A5C2D2_9',
 'Cardiac_A5

In [11]:
for (imgfile, lblfile, pid) in zip(imgs_volume_resampled, segs_volume_resampled, pids_volume_resampled):
    lblload = nib.load(lblfile)
    lbl = lblload.get_fdata()
    Imgload = nib.load(imgfile)
    Imgc1 = Imgload.get_fdata() 

#             affine_origin = Imgload.affine
    affine_origin = np.eye(4)

    newLabelImg = nib.Nifti1Image(Imgc1, affine_origin)
    imgsavepath = datasavepath + 'Image_wohdr/' + pid + '.nii.gz'
    nib.save(newLabelImg, imgsavepath)

    newLabelLbl = nib.Nifti1Image(lbl, affine_origin)
    lblsavepath = datasavepath + 'Label_wohdr/' + pid + '.nii.gz'
    nib.save(newLabelLbl, lblsavepath)
    
    print('Case ' + pid + ' saved')

Case Cardiac_A0S9V9_0 saved
Case Cardiac_A0S9V9_9 saved
Case Cardiac_A1D0Q7_0 saved
Case Cardiac_A1D0Q7_9 saved
Case Cardiac_A1D9Z7_11 saved
Case Cardiac_A1D9Z7_22 saved
Case Cardiac_A1E9Q1_0 saved
Case Cardiac_A1E9Q1_9 saved
Case Cardiac_A1K2P5_11 saved
Case Cardiac_A1K2P5_33 saved
Case Cardiac_A1O8Z3_10 saved
Case Cardiac_A1O8Z3_23 saved
Case Cardiac_A2C0I1_0 saved
Case Cardiac_A2C0I1_7 saved
Case Cardiac_A2E3W4_23 saved
Case Cardiac_A2E3W4_8 saved
Case Cardiac_A2H5K9_29 saved
Case Cardiac_A2H5K9_8 saved
Case Cardiac_A2L1N6_0 saved
Case Cardiac_A2L1N6_12 saved
Case Cardiac_A2N8V0_0 saved
Case Cardiac_A2N8V0_9 saved
Case Cardiac_A3B7E5_12 saved
Case Cardiac_A3B7E5_29 saved
Case Cardiac_A3H1O5_0 saved
Case Cardiac_A3H1O5_12 saved
Case Cardiac_A3H5R1_24 saved
Case Cardiac_A3H5R1_6 saved
Case Cardiac_A3P9V7_13 saved
Case Cardiac_A3P9V7_27 saved
Case Cardiac_A4A8V9_0 saved
Case Cardiac_A4A8V9_10 saved
Case Cardiac_A4B5U4_0 saved
Case Cardiac_A4B5U4_10 saved
Case Cardiac_A4B9O6_0 saved
Cas

Case Cardiac_D9L1Z3_0 saved
Case Cardiac_D9L1Z3_12 saved
Case Cardiac_E0J2Z9_11 saved
Case Cardiac_E0J2Z9_25 saved
Case Cardiac_E0J7L9_0 saved
Case Cardiac_E0J7L9_9 saved
Case Cardiac_E0M3U7_0 saved
Case Cardiac_E0M3U7_9 saved
Case Cardiac_E0O0S0_0 saved
Case Cardiac_E0O0S0_11 saved
Case Cardiac_E1L7M3_1 saved
Case Cardiac_E1L7M3_12 saved
Case Cardiac_E1L8Y4_10 saved
Case Cardiac_E1L8Y4_24 saved
Case Cardiac_E3F2U7_0 saved
Case Cardiac_E3F2U7_7 saved
Case Cardiac_E3F5U2_24 saved
Case Cardiac_E3F5U2_8 saved
Case Cardiac_E3I4V1_24 saved
Case Cardiac_E3I4V1_7 saved
Case Cardiac_E3L8U8_29 saved
Case Cardiac_E3L8U8_9 saved
Case Cardiac_E3T0Z2_10 saved
Case Cardiac_E3T0Z2_29 saved
Case Cardiac_E4H7L4_10 saved
Case Cardiac_E4H7L4_28 saved
Case Cardiac_E4I9O7_10 saved
Case Cardiac_E4I9O7_29 saved
Case Cardiac_E4M2Q7_0 saved
Case Cardiac_E4M2Q7_8 saved
Case Cardiac_E4O8P3_12 saved
Case Cardiac_E4O8P3_29 saved
Case Cardiac_E4W8Z7_10 saved
Case Cardiac_E4W8Z7_29 saved
Case Cardiac_E5E6O8_10 saved

Case Cardiac_L8N7Z0_9 saved
Case Cardiac_M0P8U8_0 saved
Case Cardiac_M0P8U8_9 saved
Case Cardiac_M1R4S1_0 saved
Case Cardiac_M1R4S1_9 saved
Case Cardiac_M2P1R1_0 saved
Case Cardiac_M2P1R1_10 saved
Case Cardiac_M2P5T8_0 saved
Case Cardiac_M2P5T8_11 saved
Case Cardiac_M4P7Q6_13 saved
Case Cardiac_M4P7Q6_2 saved
Case Cardiac_M4T4V6_12 saved
Case Cardiac_M4T4V6_29 saved
Case Cardiac_M6M9N1_11 saved
Case Cardiac_M6M9N1_24 saved
Case Cardiac_M6V2Y0_0 saved
Case Cardiac_M6V2Y0_9 saved
Case Cardiac_M8N3Z3_13 saved
Case Cardiac_M8N3Z3_24 saved
Case Cardiac_N1P8Q9_10 saved
Case Cardiac_N1P8Q9_29 saved
Case Cardiac_N1S7Z2_24 saved
Case Cardiac_N1S7Z2_8 saved
Case Cardiac_N2O7U5_18 saved
Case Cardiac_N2O7U5_7 saved
Case Cardiac_N5S7Y1_0 saved
Case Cardiac_N5S7Y1_9 saved
Case Cardiac_N7P3T8_10 saved
Case Cardiac_N7P3T8_25 saved
Case Cardiac_N7V9W9_24 saved
Case Cardiac_N7V9W9_7 saved
Case Cardiac_N7W6Z8_11 saved
Case Cardiac_N7W6Z8_23 saved
Case Cardiac_N8N9U0_0 saved
Case Cardiac_N8N9U0_11 saved
C

## Step 2, Intensity Normalization

In [13]:
IMG_FOLDER_SAVE = datasavepath + 'Image_wohdr/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [14]:
imgs

['./datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/Image_wohdr/Cardiac_A2C0I1_7.nii.gz',
 './dataprepro

In [15]:
for (imgfile, pid) in zip(imgs, pids_volume_resampled):
    savename = pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    
    array = sitk.GetArrayFromImage(img_obj)

    pixel_mean = np.mean(array)
    pixel_std = np.std(array)
    array = (array - pixel_mean) / pixel_std

    normalized_img = sitk.GetImageFromArray(array)
    normalized_img.CopyInformation(img_obj)
    
    sitk.WriteImage(normalized_img, datasavepath + 'processed/' + savename, True) 
    print('Id ' + pid + ' normalized')

Id Cardiac_A0S9V9_0 normalized
Id Cardiac_A0S9V9_9 normalized
Id Cardiac_A1D0Q7_0 normalized
Id Cardiac_A1D0Q7_9 normalized
Id Cardiac_A1D9Z7_11 normalized
Id Cardiac_A1D9Z7_22 normalized
Id Cardiac_A1E9Q1_0 normalized
Id Cardiac_A1E9Q1_9 normalized
Id Cardiac_A1K2P5_11 normalized
Id Cardiac_A1K2P5_33 normalized
Id Cardiac_A1O8Z3_10 normalized
Id Cardiac_A1O8Z3_23 normalized
Id Cardiac_A2C0I1_0 normalized
Id Cardiac_A2C0I1_7 normalized
Id Cardiac_A2E3W4_23 normalized
Id Cardiac_A2E3W4_8 normalized
Id Cardiac_A2H5K9_29 normalized
Id Cardiac_A2H5K9_8 normalized
Id Cardiac_A2L1N6_0 normalized
Id Cardiac_A2L1N6_12 normalized
Id Cardiac_A2N8V0_0 normalized
Id Cardiac_A2N8V0_9 normalized
Id Cardiac_A3B7E5_12 normalized
Id Cardiac_A3B7E5_29 normalized
Id Cardiac_A3H1O5_0 normalized
Id Cardiac_A3H1O5_12 normalized
Id Cardiac_A3H5R1_24 normalized
Id Cardiac_A3H5R1_6 normalized
Id Cardiac_A3P9V7_13 normalized
Id Cardiac_A3P9V7_27 normalized
Id Cardiac_A4A8V9_0 normalized
Id Cardiac_A4A8V9_10 nor

Id Cardiac_D3O9U9_10 normalized
Id Cardiac_D3Q0W9_10 normalized
Id Cardiac_D3Q0W9_24 normalized
Id Cardiac_D4M3Q2_0 normalized
Id Cardiac_D4M3Q2_9 normalized
Id Cardiac_D4N6W6_0 normalized
Id Cardiac_D4N6W6_10 normalized
Id Cardiac_D5G3W8_23 normalized
Id Cardiac_D5G3W8_8 normalized
Id Cardiac_D6E9U8_0 normalized
Id Cardiac_D6E9U8_12 normalized
Id Cardiac_D6H6O2_29 normalized
Id Cardiac_D6H6O2_9 normalized
Id Cardiac_D6N7Q8_24 normalized
Id Cardiac_D6N7Q8_9 normalized
Id Cardiac_D7M8P9_24 normalized
Id Cardiac_D7M8P9_8 normalized
Id Cardiac_D7T3V8_24 normalized
Id Cardiac_D7T3V8_7 normalized
Id Cardiac_D8E4F4_0 normalized
Id Cardiac_D8E4F4_10 normalized
Id Cardiac_D8O0W2_24 normalized
Id Cardiac_D8O0W2_8 normalized
Id Cardiac_D9F5P1_0 normalized
Id Cardiac_D9F5P1_10 normalized
Id Cardiac_D9I8O7_11 normalized
Id Cardiac_D9I8O7_29 normalized
Id Cardiac_D9L1Z3_0 normalized
Id Cardiac_D9L1Z3_12 normalized
Id Cardiac_E0J2Z9_11 normalized
Id Cardiac_E0J2Z9_25 normalized
Id Cardiac_E0J7L9_0 n

Id Cardiac_J8R5W2_9 normalized
Id Cardiac_J9L4S2_13 normalized
Id Cardiac_J9L4S2_29 normalized
Id Cardiac_J9L6N9_29 normalized
Id Cardiac_J9L6N9_9 normalized
Id Cardiac_K2S1U6_12 normalized
Id Cardiac_K2S1U6_29 normalized
Id Cardiac_K3P3Y6_11 normalized
Id Cardiac_K3P3Y6_32 normalized
Id Cardiac_K3R0Y7_28 normalized
Id Cardiac_K3R0Y7_9 normalized
Id Cardiac_K4T7Y0_12 normalized
Id Cardiac_K4T7Y0_24 normalized
Id Cardiac_K5K6N1_24 normalized
Id Cardiac_K5K6N1_9 normalized
Id Cardiac_K5L2U3_0 normalized
Id Cardiac_K5L2U3_7 normalized
Id Cardiac_K5L4S1_10 normalized
Id Cardiac_K5L4S1_27 normalized
Id Cardiac_K5M7V5_10 normalized
Id Cardiac_K5M7V5_23 normalized
Id Cardiac_K5P0Y1_0 normalized
Id Cardiac_K5P0Y1_9 normalized
Id Cardiac_K6N4N7_0 normalized
Id Cardiac_K6N4N7_8 normalized
Id Cardiac_K7L2Y6_10 normalized
Id Cardiac_K7L2Y6_29 normalized
Id Cardiac_K7N0R7_29 normalized
Id Cardiac_K7N0R7_9 normalized
Id Cardiac_K7O3Q0_10 normalized
Id Cardiac_K7O3Q0_23 normalized
Id Cardiac_K9N0W0_1

## Step 3, Generate Body Masks

In [16]:
IMG_FOLDER_SAVE = datasavepath + 'Image_wohdr/'
imgs = glob.glob(IMG_FOLDER_SAVE + "/*.nii.gz")
imgs = [ fid for fid in sorted(imgs) ]

In [19]:
for (imgfile, pid) in zip(imgs, pids_volume_resampled):
    savename = pid + '.nii.gz'
    img_obj = sitk.ReadImage( imgfile )
    array = sitk.GetArrayFromImage(img_obj)
    array_mask = array > -999
    array_mask = array_mask.astype(float)

    mask_img = sitk.GetImageFromArray(array_mask)
    mask_img.CopyInformation(img_obj)
    
    sitk.WriteImage(mask_img, datasavepath + 'mask/' + savename, True) 

## Generate datafiles.

In [49]:
datafilepath = './datafiles/MnMCardiac/'
datapreprocessedpath = datasavepath + 'processed/'

In [30]:
if os.path.exists(datafilepath) == False:
    os.mkdir(datafilepath)
    os.mkdir(datafilepath + 'datafiletrainingSource')
    os.mkdir(datafilepath + 'datafilevalSourceTarget')
    os.mkdir(datafilepath + 'datafiletestSource')
    os.mkdir(datafilepath + 'datafiletestTarget')

### training as the folder.

In [37]:
Imgdatafiles = datafilepath + 'datafiletrainingSource/Imgpre-train.txt'
Lbldatafiles = datafilepath + 'datafiletrainingSource/seg-train.txt'
Maskfiles = datafilepath + 'datafiletrainingSource/mask-train.txt'

In [45]:
imgs_train = glob.glob(datadownloadpath_Train + "*/*/*_sa.nii.gz")
imgs_train = [ fid for fid in sorted(imgs_train) ]
pids = [pid.split("/")[-1].split("_")[0] for pid in imgs_train]

In [48]:
pids

['A0S9V9',
 'A1D0Q7',
 'A1D9Z7',
 'A1E9Q1',
 'A1O8Z3',
 'A2C0I1',
 'A2N8V0',
 'A3B7E5',
 'A3H1O5',
 'A4B5U4',
 'A4J4S4',
 'A4U9V5',
 'A5E0T8',
 'A6B5G9',
 'A6D5F9',
 'A6M1Q7',
 'A7D9L8',
 'A7G0P5',
 'A7M7P8',
 'A7O4T6',
 'A8C9U8',
 'A8E1F4',
 'A9C5P4',
 'A9E3G9',
 'A9J5Q7',
 'A9J8W7',
 'B0I2Z0',
 'B0N3W8',
 'B2C2Z7',
 'B2D9M2',
 'B2D9O2',
 'B2F4K5',
 'B2G5R2',
 'B3D0N1',
 'B3O1S0',
 'B3P3R1',
 'B4O3V3',
 'B6D0U7',
 'B8H5H6',
 'B8J7R4',
 'B9E0Q1',
 'B9O1Q0',
 'C0K1P0',
 'C0S7W0',
 'C1G5Q0',
 'C1K8P5',
 'C2J0K3',
 'C2L5P7',
 'C2M6P8',
 'C3I2K3',
 'C4R8T7',
 'C4S8W9',
 'C5M4S2',
 'C6J5P1',
 'C8P3S7',
 'D0H9I4',
 'D0R0R9',
 'D1J5P6',
 'D1L4Q9',
 'D1M1S6',
 'D3D4Y5',
 'D3F3O5',
 'D3F9H9',
 'D3O9U9',
 'D4M3Q2',
 'D4N6W6',
 'D6H6O2',
 'D8E4F4',
 'D9L1Z3',
 'E0M3U7',
 'E0O0S0',
 'E3T0Z2',
 'E4M2Q7',
 'E4W8Z7',
 'E5E6O8',
 'E5F5V7',
 'E9H1U4',
 'E9H2K7',
 'E9L1W5',
 'E9V4Z8',
 'F0J2R8',
 'F1F3I6',
 'F2H5S1',
 'F3G5K5',
 'F4K3S1',
 'F5I9Q2',
 'F8N2S1',
 'G0H4J3',
 'G0I6P3',
 'G1N6S7',
 'G2J1M5',

In [50]:
imgsprocessed_train = glob.glob(datapreprocessedpath + "*.nii.gz")
imgsprocessed_train = [ fid for fid in sorted(imgsprocessed_train) ]

In [51]:
imgsprocessed_train

['./datapreprocessed/MnMCardiac/processed/Cardiac_A0S9V9_0.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A0S9V9_9.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1D0Q7_0.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1D0Q7_9.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1D9Z7_11.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1D9Z7_22.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1E9Q1_0.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1E9Q1_9.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1K2P5_11.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1K2P5_33.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1O8Z3_10.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A1O8Z3_23.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A2C0I1_0.nii.gz',
 './datapreprocessed/MnMCardiac/processed/Cardiac_A2C0I1_7.nii.gz',
 './datapreprocessed/MnMCardiac/processed/

In [61]:
# write image files
with open(Imgdatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write(filename)
            f.write('\n')
# write label files
with open(Lbldatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write('./datapreprocessed/MnMCardiac/Label_wohdr/' + filename.split("/")[-1])
            f.write('\n')
# write mask files
with open(Maskfiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write('./datapreprocessed/MnMCardiac/mask/' + filename.split("/")[-1])
            f.write('\n')

### validation as the folder.

In [62]:
Imgdatafiles = datafilepath + 'datafilevalSourceTarget/Imgpre-train.txt'
Lbldatafiles = datafilepath + 'datafilevalSourceTarget/seg-train.txt'
Maskfiles = datafilepath + 'datafilevalSourceTarget/mask-train.txt'

In [63]:
# just use this split.
imgs_val = glob.glob(datadownloadpath_Val + "*/*_sa.nii.gz")
imgs_val = [ fid for fid in sorted(imgs_val) ]
pids = [pid.split("/")[-1].split("_")[0] for pid in imgs_val]

In [65]:
len(pids)

34

In [66]:
# write image files
with open(Imgdatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write(filename)
            f.write('\n')
# write label files
with open(Lbldatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write('./datapreprocessed/MnMCardiac/Label_wohdr/' + filename.split("/")[-1])
            f.write('\n')
# write mask files
with open(Maskfiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids:
            f.write('./datapreprocessed/MnMCardiac/mask/' + filename.split("/")[-1])
            f.write('\n')

### test as the folder, note the source/target differences.

In [68]:
# test source data, as the folder.
Imgdatafiles = datafilepath + 'datafiletestSource/Imgpre-eval.txt'
Lbldatafiles = datafilepath + 'datafiletestSource/seg-eval.txt'
Maskfiles = datafilepath + 'datafiletestSource/mask-eval.txt'

In [69]:
# just use this split.
imgs_test = glob.glob(datadownloadpath_Test + "*/*_sa.nii.gz")
imgs_test = [ fid for fid in sorted(imgs_test) ]
pids = [pid.split("/")[-1].split("_")[0] for pid in imgs_test]

In [70]:
len(pids)

136

In [85]:
metaload = pd.read_csv(metadata)
metaload_group = metaload.loc[metaload['Centre']==5]

In [86]:
# write image files
with open(Imgdatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] not in metaload_group['External code'].values:
            f.write(filename)
            f.write('\n')
# write label files
with open(Lbldatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] not in metaload_group['External code'].values:
            f.write('./datapreprocessed/MnMCardiac/Label_wohdr/' + filename.split("/")[-1])
            f.write('\n')
# write mask files
with open(Maskfiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] not in metaload_group['External code'].values:
            f.write('./datapreprocessed/MnMCardiac/mask/' + filename.split("/")[-1])
            f.write('\n')

In [87]:
# test target data, as the folder.
Imgdatafiles = datafilepath + 'datafiletestTarget/Imgpre-eval.txt'
Lbldatafiles = datafilepath + 'datafiletestTarget/seg-eval.txt'
Maskfiles = datafilepath + 'datafiletestTarget/mask-eval.txt'

In [88]:
# write image files
with open(Imgdatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] in metaload_group['External code'].values:
            f.write(filename)
            f.write('\n')
# write label files
with open(Lbldatafiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] in metaload_group['External code'].values:
            f.write('./datapreprocessed/MnMCardiac/Label_wohdr/' + filename.split("/")[-1])
            f.write('\n')
# write mask files
with open(Maskfiles, 'w') as f:
    for filename in imgsprocessed_train: 
        if filename.split("/")[-1].split("_")[1] in pids and filename.split("/")[-1].split("_")[1] in metaload_group['External code'].values:
            f.write('./datapreprocessed/MnMCardiac/mask/' + filename.split("/")[-1])
            f.write('\n')